In [ ]:
import inspect

In [ ]:
from ray.rllib.utils.metrics import ENV_RUNNER_RESULTS, EPISODE_RETURN_MEAN

from flatland.ml.ray.examples.flatland_inference_with_random_policy import add_flatland_inference_with_random_policy_args, rollout
from flatland.ml.ray.examples.flatland_training_with_parameter_sharing import train, add_flatland_training_with_parameter_sharing_args, \
    register_flatland_ray_cli_observation_builders

In [ ]:
# Rllib Demo

### Register observation builds in rllib input registry

These are the registered keys you can use for the `--obs-builder` param below. Use `regiser_input` to register your own.

In [ ]:
%pycat inspect.getsource(register_flatland_ray_cli_observation_builders)

In [ ]:
register_flatland_ray_cli_observation_builders()

## Rllib Training

In [ ]:
parser = add_flatland_training_with_parameter_sharing_args()

#### Inspect Training cli Options

In [ ]:
!python -m flatland.ml.ray.examples.flatland_training_with_parameter_sharing --help

#### Inspect Training cli Code

In [ ]:
%pycat inspect.getsource(train)

#### Run Training with PPO for one iteration with reduced batch size and checkpointing

In [ ]:
!echo $PWD

In [ ]:
algo = "PPO"
obid = "FlattenedNormalizedTreeObsForRailEnv_max_depth_3_50"
# in order to get the results, we call `train()` directly from python
results = train(parser.parse_args(
    ["--num-agents", "2", "--obs-builder", obid, "--algo", algo, "--stop-iters", "1", "--train-batch-size-per-learner", "200", "--checkpoint-freq", "1"]))

## Rollout from Checkpoint

In [ ]:
parser = add_flatland_inference_with_random_policy_args()

#### Inspect Rollout cli Options

In [ ]:
!python -m flatland.ml.ray.examples.flatland_inference_with_random_policy --help

#### Inspect Rollout cli Code

In [ ]:
%pycat inspect.getsource(rollout)

#### Rollout on best checkpoint from previous training

In [ ]:
register_flatland_ray_cli_observation_builders()

In [ ]:
best_result = results.get_best_result(
    metric=f"{ENV_RUNNER_RESULTS}/{EPISODE_RETURN_MEAN}", mode="max"
)

In [ ]:
!python -m flatland.ml.ray.examples.flatland_inference_with_random_policy --num-agents 2 --obs-builder {obid} --cp {best_result.checkpoint.path} --policy-id p0  --num-episodes-during-inference 1